# Chapter 12 Setting the stage: Preparing features for machine learning
we will disucss
- How investing in a solid data manipulation foundation makes data preparation a breeze
- Addressing big data quality problems with PySpark
- Creating custom features for your ML model
- Selecting compelling features for your model
- Using transformers and estimators as part of the feature engineering process

## Main Step for Data Cleaning and Pro-processing

- standardizing column names
- check data types and correct mismatching data type
- deal with missing values
- deal with outliers
- deal with correlations among features
- deal with the feaures with raw occurance
- standarize/normalize features
- feature creation/engineering
- fearure refinement

In [ ]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F
import pyspark.sql.types as T
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# change the account name to your email account
account='sli'

# define a root path to access the data in the DataAnalysisWithPythonAndPySpark
data_path='/net/clusterhn/home/'+account+'/isa460/data/'

# append path to helper_functions to system path
sys.path.append('/net/clusterhn/home/sli/isa460_sli')

import helper_functions as H

# check if the Spark session is active. If it is activate, close it

try:
    if spark:
        spark.stop()
except:
    pass    

spark = (SparkSession.builder.appName("Preparing Featurers for Machine Learning")
        .config("spark.port.maxRetries", "100")
        .config("spark.sql.mapKeyDedupPolicy", "LAST_WIN")  # This configuration allow the duplicate keys in the map data type.
        .config("spark.driver.memory", "8g")
        .getOrCreate())

# confiture the log level (defaulty is WARN)
spark.sparkContext.setLogLevel('ERROR')

# Reading, exploring, and preparing our machine learning data set
For our ML model, we chose a data set of 20,057 dish names that contain 680 columns characterizing the ingredient list, the nutritional content, and the category of the dish. Our goal here is to predict if this dish is a dessert.

In [ ]:
# load data
food=spark.read.csv(data_path+'recipes/epi_r.csv', inferSchema=True, header=True)

In [ ]:
print(food.count(), len(food.columns))

In [ ]:
food.select('title',
 'rating',
 'calories',
 'protein',
 'fat',
 'sodium',
 '#cakeweek',
 '#wasteless',
'dessert').show()

In [ ]:
food.groupBy('dessert').count().show()

In [ ]:
#food.printSchema()

## Standardizing column names using toDF()
We will remove anything that isn’t a letter or a number, standardize the spaces and other separators to use the underscore (_) character, and replace the ampersand (&) with its English equivalent and.

In [ ]:
def sanitize_column_name(name):
    """Drops unwanted characters from the column name.
 
    We replace spaces, dashes and slashes with underscore,
    and only keep alphanumeric characters.
    """
    answer = name
    for i, j in ((" ", "_"), ("-", "_"), ("/", "_"), ("&", "and")):   
        answer = answer.replace(i, j)
    return "".join(
        [
            char
            for char in answer
            if char.isalpha() or char.isdigit() or char == "_"       
        ]
    )
# use toDF() to apply functions to all columns in the data frame
food1 = food.toDF(*[sanitize_column_name(name) for name in food.columns])

In [ ]:
#food1.printSchema()

In [ ]:
# Exercise: Modify the above functions to change all characters to lower case
# and apply to the food data frame

def sanitize_column_name(name):
    """Drops unwanted characters from the column name.
 
    We replace spaces, dashes and slashes with underscore,
    and only keep alphanumeric characters.
    """
    answer = name.lower()
    for i, j in ((" ", "_"), ("-", "_"), ("/", "_"), ("&", "and")):   
        answer = answer.replace(i, j)
    return "".join(
        [
            char
            for char in answer
            if char.isalpha() or char.isdigit() or char == "_"       
        ]
    )
# use toDF() to apply functions to all columns in the data frame
food1 = food.toDF(*[sanitize_column_name(name) for name in food.columns])

In [ ]:
#food1.printSchema()

## Exploring our data and getting our first feature columns

In [ ]:
# check the statistics of numerical columns. As an example, we only focuse on the first 20 columns
for x in food.columns[0:20]:
    if food.schema[x].dataType!=T.StringType():
        food.select(x).summary().show()

In [ ]:
food.groupBy('anise').count().show()

In [ ]:
# check specific column

food.select('clove').summary().show()

In [ ]:
# check all columns with binary value. As an example, we will only focus on the first 20 columns.
import pandas as pd
from pyspark.sql import functions as F

is_binary = food1.agg(
    *[(F.countDistinct(col) == 2).alias(col) for col in food1.columns[0:20]]
).toPandas()

In [ ]:
#is_binary
is_binary.unstack()

## Addressing data misshaps, using cakeweek and wasterless as example

In [ ]:
food1.groupBy('cakeweek').count().show()

In [ ]:
food1.groupBy('wasteless').count().show()

In [ ]:
# for cakeweek and wasteless, only keep 0, 1 and null. Drop the rest

food2 = food1.where(
    (
        F.col("cakeweek").isin([0.0, 1.0])    
        | F.col("cakeweek").isNull()         
    )
    & (
        F.col("wasteless").isin([0.0, 1.0])    
        | F.col("wasteless").isNull()         
    )
)
 
print(food1.count(), food2.count())
 
# result 20057 20054. Three records were removed.                   

In [ ]:
# Check category columns
catColumns=[x for x in food2.columns if food2.schema[x].dataType==F.StringType()]

catColumns

In [ ]:
# rating and calories should be double. Covert the type from string to double

food2=food2.withColumn('rating', F.col('rating').cast('Double')) \
     .withColumn('calories', F.col('calories').cast('Double'))

### Create four top level variables

In [ ]:
IDENTIFIERS = ["title"]
 
CONTINUOUS_COLUMNS = [
    "rating",
    "calories",
    "protein",
    "fat",
    "sodium",
]
 
TARGET_COLUMN = ["dessert"]
 
BINARY_COLUMNS = [
    x
    for x in food2.columns
    if x not in CONTINUOUS_COLUMNS
    and x not in TARGET_COLUMN
    and x not in IDENTIFIERS
]

## Deal with missing values (Weeding our useless records and imputing binary features)

In [ ]:
# drop records where all features are null

food3 = food2.dropna(
    how="all",
    subset=[x for x in food2.columns if x not in IDENTIFIERS]
)

# drop records whereh target column is null

food3 = food3.dropna(subset=TARGET_COLUMN)

# impute binary columns. Fill null value with 0

food3=food3.fillna(0.0, subset=BINARY_COLUMNS)

## Take care of extreme values/outliners

In [ ]:
food3.select(*CONTINUOUS_COLUMNS).summary().show()

In [ ]:
# return 99 percentile of each feature

selected_columns=[ "calories", "protein", "fat","sodium"]

maximum={}

for c in selected_columns:
    maximum[c]=food3.select(F.percentile_approx(c, 0.99)).collect()[0][0]
    
maximum

In [ ]:
# for the above feature, replace any value over 99 percentile to the value at 99 percentile

for k, v in maximum.items():
    food3=food3.withColumn(k, F.when(F.isnull(F.col(k)), F.col(k)).otherwise(
        F.least(F.col(k), F.lit(v))
    ))

In [ ]:
### Check the boxplot of each feature

selected_columns=[ "calories", "protein", "fat","sodium"]

# without removing outliners
#df=food2.select(selected_columns).toPandas()

# with outliners removed
df=food3.select(selected_columns).toPandas()

# Set up a 4X1 grid of subplots
fig, axes = plt.subplots(4, 1, figsize=(8, 14))

i=0
for c in selected_columns:
    sns.boxplot(x=df[c], ax=axes[i])
    i=i+1
    
# Adjust layout
plt.tight_layout()
plt.show()

## Weeding out the rare binary occurrence columns

Binary features with only a few zeroes or ones are not helpful in classifying a recipe as a dessert: if every recipe (or no recipe) has a certain feature as true, then that feature does not discriminate properly, meaning that our model has no use for it.

In [ ]:
# for binary variables, remove features with less then 10 of 0 or 1.

inst_sum_of_binary_columns = [
    F.sum(F.col(x)).alias(x) for x in BINARY_COLUMNS
]
 
sum_of_binary_columns = (
    food3.select(*inst_sum_of_binary_columns).head().asDict()         
)
 
num_rows = food3.count()
too_rare_features = [
    k
    for k, v in sum_of_binary_columns.items()
    if v < 10 or v > (num_rows - 10)
]

BINARY_COLUMNS = list(set(BINARY_COLUMNS) - set(too_rare_features)) 

In [ ]:
#too_rare_features

len(too_rare_features)

# Feature Creation/Engineering and Refinement

- Creating a few custom features using our continuous feature columns

- Measuring correlation over original and generated continuous features

## Feaure Creation
As an example, we’ll take the protein and fat columns representing the quantity (in grams) of protein and fat in the recipe, respectively. With the information in those two columns, we create two features representing the percentage of calories attributed to each macro nutriment.

In [ ]:
food3 = food3.withColumn(
    "protein_ratio", F.col("protein") * 4 / F.col("calories")
).withColumn(
    "fat_ratio", F.col("fat") * 9 / F.col("calories")
)                                                           
 
food3 = food3.fillna(0.0, subset=["protein_ratio", "fat_ratio"])
 
CONTINUOUS_COLUMNS += ["protein_ratio", "fat_ratio"]    

## Removing highly correlated features

We want our features to be correlated with our target (this provides predictive power). On the other hand, we want to avoid correlation between features for two main reasons:
- If two features are highly correlated, it means that they provide almost the same information. In the context of machine learning, this can confuse the fitting algorithm and create model or numerical instability.
- The more complex your model, the more complex the maintenance. Highly correlated features rarely provide improved accuracy, yet complicate the model. Simple is better.

### convert to pandas to calculate correlation

In [ ]:
corr=food3.select(CONTINUOUS_COLUMNS).toPandas().corr()
corr

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')

### Use pyspark way to calcualte correlation

In [ ]:
from pyspark.ml.feature import VectorAssembler
 
continuous_features = VectorAssembler(
    inputCols=CONTINUOUS_COLUMNS, outputCol="continuous_features"
)
 
vector_food = food3.select(CONTINUOUS_COLUMNS)
for x in CONTINUOUS_COLUMNS:
    vector_food = vector_food.where(~F.isnull(F.col(x))) 
 
vector_variable = continuous_features.transform(vector_food)
 
vector_variable.select("continuous_features").show(3, False)

In [ ]:
from pyspark.ml.stat import Correlation
 
correlation = Correlation.corr(
    vector_variable, "continuous_features"                
)
 
correlation_array = correlation.head()[0].toArray()       
 
correlation_pd = pd.DataFrame(
    correlation_array,                                      
    index=CONTINUOUS_COLUMNS,                               
    columns=CONTINUOUS_COLUMNS,                             
)

In [ ]:
print(correlation_pd.iloc[:, :4])

## Feature preparation with transformers and estimators
We explore two relevant examples of transformers and estimators:
- Null imputation, where we provide a value to replace null occurrences in a column (e.g., the mean)
- Scaling features, where we normalize the values of a column, so they are on a more logical scale (e.g., between zero and one)

[Function Approach and Transformer Approach](https://raw.githubusercontent.com/Suhong88/ISA460_Fall2023/main/images/Figure%2012.2.PNG)

### Imputing continuous features using the Imputer estimator

At the core, an estimator is a transformer-generating object. We instantiate an estimator object just like a transformer by providing the relevant parameters to its constructor. To apply an estimator, we call the fit() method, which returns a Model object, which is, for all purposes, the same as a transformer. Estimators allow for automatically creating transformers that depend on the data

[Imputer and its companion transformer/model](https://raw.githubusercontent.com/Suhong88/ISA460_Fall2023/main/images/Figure%2012.3.JPG)

In [ ]:
# impute the following 5 featurs with mean

from pyspark.ml.feature import Imputer
 
OLD_COLS = ["rating", "calories", "protein", "fat", "sodium"]
NEW_COLS = ["rating_i", "calories_i", "protein_i", "fat_i", "sodium_i"]
 
imputer = Imputer(
    strategy="mean",                                            
    inputCols=OLD_COLS,                                         
    outputCols=NEW_COLS,                                        
)
 
imputer_model = imputer.fit(food3)  

food_imputed = imputer_model.transform(food3)
 
CONTINUOUS_COLUMNS = (
    list(set(CONTINUOUS_COLUMNS) - set(OLD_COLS)) + NEW_COLS    
)

CONTINUOUS_COLUMNS=set(CONTINUOUS_COLUMNS)

### Scaling our features using the MinMaxScaler estimator

In [ ]:
from pyspark.ml.feature import MinMaxScaler
 
CONTINUOUS_NB = [x for x in CONTINUOUS_COLUMNS if "ratio" not in x]
 
continuous_assembler = VectorAssembler(
    inputCols=CONTINUOUS_NB, outputCol="continuous"
)
 
food_features = continuous_assembler.transform(food_imputed)
 
continuous_scaler = MinMaxScaler(
    inputCol="continuous",
    outputCol="continuous_scaled",
)
 
food_features = continuous_scaler.fit(food_features).transform(food_features)

food_features.select("continuous_scaled").show(3, False)

# Summary

- big part of creating a machine learning model is data manipulation. For this, everything we’ve learned within pyspark.sql can be leveraged.
- The first step in creating an ML model is assessing data quality and addressing potential data problems. Moving from large data sets in PySpark to small summaries in pandas or plain Python speeds up data discovery and assessment.
- Feature creation and selection can either be done manually using the PySpark data manipulation API or by leveraging some pyspark.ml-specific constructors, such as the correlation matrix.
- Multiple common feature engineering patterns, like imputing and scaling features, are provided through PySpark transformers and estimators.